In [7]:
# %pip install --upgrade langchain langsmith langgraph langchain_openai

In [8]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OpenAI keys from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Agent Demo"

### Set up the Tools


In [9]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

In [10]:
class MultiplierInput(BaseModel):
    a: int = Field(description="First number")
    b: int = Field(description="Second number")


def multiply(a: int, b: int) -> int:
    return a * b


multiplier = StructuredTool.from_function(
    func=multiply,
    name="Multiplier",
    description="Multiply two numbers",
    args_schema=MultiplierInput,
    return_direct=False,
)

In [11]:
class AdderInput(BaseModel):
    a: int = Field(description="First number")
    b: int = Field(description="Second number")


def add(a: int, b: int) -> int:
    return a + b


adder = StructuredTool.from_function(
    func=add,
    name="Adder",
    description="Add two numbers",
    args_schema=AdderInput,
    return_direct=False,
)

In [12]:
tools = [multiplier, adder]

In [13]:
# Modification: we need to be able to execute tools from the graph
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

### Set up the agent


In [14]:
# Modification: we're defining the resposne schema for our agent
from langchain_core.pydantic_v1 import BaseModel, Field


class Response(BaseModel):
    """Final answer to the user"""

    result: int = Field(description="the result of the computation")
    explanation: str = Field(
        description="explanation of the steps taken to get the result"
    )

In [15]:
# Modification: instead of initializing a full Agent Executor, we're just creating the model and banding functions to it
from langchain_openai import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

# Create the OpenAI LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)

# Bind tools to the model
functions = [format_tool_to_openai_function(t) for t in tools]
# Bind the resposne to the model
functions.append(convert_pydantic_to_openai_function(Response))

model = llm.bind_functions(functions)

### Set up the Agent State
Everything from here onwards is new

In [16]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

### Set up the node actions


In [17]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage


# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    elif last_message.additional_kwargs["function_call"]["name"] == "Response":
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the function to execute tools
def call_tool(state):
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )

    # response = input(prompt=f"[y/n] continue with: {action}?")
    # if response == "n":
    #     raise ValueError("User cancelled")
    
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

### Define the Graph


In [18]:
from langgraph.graph import StateGraph, END

# Initialize a new graph
graph = StateGraph(AgentState)

# Define the two "Nodes"" we will cycle between
graph.add_node("agent", call_model)
graph.add_node("action", call_tool)

# Define all our "Edges"
# Set the "Starting Edge" as "agent"
# This means that this node is the first one called
graph.set_entry_point("agent")

# We now add a "Conditional Edge"
# Conditinal agents take:
# - A start node
# - A function that determines which node to call next
# - A mapping of the output of the function to the next node to call
graph.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        # END is a special node marking that the graph should finish.
        "end": END,
    },
)

# We now add a "Normal Edge" that should always be called after another
graph.add_edge("action", "agent")

# We compile the entire workflow as a runnable
app = graph.compile()

### Run our graph


In [19]:
from langchain_core.messages import HumanMessage

inputs = {
    "messages": [HumanMessage(content="what is the product of 37 and 54 plus 42?")]
}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"a":37,"b":54}', 'name': 'Multiplier'}})]}

---

Output from node 'action':
---
{'messages': [FunctionMessage(content='1998', name='Multiplier')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"a":1998,"b":42}', 'name': 'Adder'}})]}

---

Output from node 'action':
---
{'messages': [FunctionMessage(content='2040', name='Adder')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"result":2040,"explanation":"The product of 37 and 54 is 1998. Adding 42 to 1998 gives a final result of 2040."}', 'name': 'Response'}})]}

---

Output from node '__end__':
---
{'messages': [HumanMessage(content='what is the product of 37 and 54 plus 42?'), AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"a":37,"b":54}',